# Global Averaging

in this notebook, intermediate layers from vgg16 will be extracted. the output will be a feature vector (ex: 512, 18,18). 512 is the kernels and 18 the width and height. For every pixel, the average of those 512 values will be computed. 

The result is a vector (512,) where every kernel will have only one value. Using the cosine similarity and the euclidean distance, it computes the flatten array of the images before and after. Results will be stored in a csv file.

In [1]:
import numpy as np
import torchvision
import pandas as pd
import glob
import os

from PIL import Image
from typing import Callable, Union
from scipy.spatial import distance

## Load model

Load the model vgg16, register the hook in a layer (ex: 30).
Open and transform the images.

In [2]:
layer = 11
path = "../data/preprocess/"
features = {}
model_name = 'vgg16'

model = torchvision.models.vgg16(pretrained=True)

def reg_hook(layer: int) -> Callable:
    def hook(model, input, output):
        features['feats'] = output.detach()
    return hook

model.features[layer].register_forward_hook(reg_hook('feats'))

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    ),
])

def open_images(path: str, case: str, situation: str) -> list:
    lst = []
    for filename in glob.glob(path + case + "/" + situation + "/*.JPG"):
        lst.append(Image.open(filename).convert("RGB"))
    return lst

def extract_features(model: torchvision.models, transform: Callable, images: list) -> list:

    # inner function to normalize the vector
    def normalize(A: np.ndarray) -> np.ndarray:
        norm = np.linalg.norm(A)
        return A / norm

    embeddings = []
    
    # iterate through all images and save the features in the list
    for image in images:
        x = transform(image).unsqueeze(0).to("cpu")
        _ = model(x)
        embeddings.append(normalize(features['feats'].cpu().numpy()))
        
    return embeddings

## Distances

for this notebook, cosine similarity and euclidean distance will be used.

In [3]:
def cosine_similarity(A: Union[np.ndarray, list], B: Union[np.ndarray, list]) -> tuple:
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

def Average(lst):
    return sum(lst) / len(lst)

def distance_metric(embeddings: list) -> float:
    
    A = embeddings[0]
    B = embeddings[1]
    C = embeddings[2]
    
    def transform_dist_to_sim(x: float) -> float:
        return 1 / (1 + x)

    e1 = transform_dist_to_sim(distance.euclidean(A, B))
    e2 = transform_dist_to_sim(distance.euclidean(A, C))

#     c1 = cosine_similarity(A, B)
#     c2 = cosine_similarity(A, C)
    

    return Average([e1, e2])

### Feature Extraction

Opening the image and extract the feature

#### Perform the global averaging
With the feature vectors from the images, uses the function perform_global_averaging to get the mean of the pixels of every kernel.

##### Result
the result is a 1D array with the size of the kernels

In [4]:
def global_averaging_on_kernel(vector: np.ndarray) -> np.ndarray:
    for i, val in enumerate(vector):
        vector[i] = np.mean(val)
    return vector[:, 0, 0]

def global_averaging_on_pixels(vector: np.ndarray) -> list:
    vector = np.transpose(vector)
    res = []
    for i in range(vector.shape[0]):
        for j in range(vector.shape[1]):
            res.append(np.mean(vector[i][j]))
    return res
            

cases = [f.path[19:] for f in os.scandir(path) if f.is_dir()]
results = []

for case in cases:
    
    # get the current case
    images_before_treatment = open_images(path=path, case=case, situation='BEFORE')
    images_after_treatment = open_images(path=path, case=case, situation='AFTER')

    # get the embeddings
    embeddings_before = extract_features(model=model, transform=transform, images=images_before_treatment)
    embeddings_after = extract_features(model=model, transform=transform, images=images_after_treatment)
    
    # perform the global averaging on the embeddings
    before_lst = [global_averaging_on_kernel(vector[0]) for vector in embeddings_before]
    after_lst = [global_averaging_on_kernel(vector[0]) for vector in embeddings_after]
    
    # calculate distances
    before_distances = distance_metric(before_lst)
    after_distances = distance_metric(after_lst)
    
    # add the results in the final list
    results.append((case, round(before_distances, 3), round(after_distances, 3)))

In [5]:
# create dataframe and store it in a csv file.
df = pd.DataFrame(results, columns=['Case', 'Before', 'After'])

# store df in a csv file.
df.to_csv(f'../csv/global_averaging/{model_name}_{layer}_global_avg_kernel_eucl.csv', index=False)

In [6]:
print(before_lst[0].shape)
print(model)

(256,)
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dil